# Bloque 6: Limpieza y Validación de Datos de Biodiversidad

En este bloque aprenderás a:
✅ Identificar errores comunes en datos de ocurrencia biológica.  
✅ Limpiar y validar coordenadas, fechas y nombres científicos.  
✅ Eliminar duplicados y manejar valores faltantes.  
✅ Crear un dataset limpio listo para análisis exploratorio o espacial.

💡 *Recuerda:* La calidad del análisis depende directamente de la calidad de los datos.


2. Importar librerías y cargar el dataset

In [ ]:
import pandas as pd
import numpy as np

# Cargar el dataset (puede ser el descargado desde GBIF)
df = pd.read_csv('/content/db_Dasypus_novemcinctus.csv')

# Vista rápida
df.head()


3. Exploración de calidad general

In [ ]:
# Tamaño y estructura
print("Filas y columnas:", df.shape)
df.info()

# Conteo de valores faltantes
df.isna().sum().sort_values(ascending=False).head(10)


In [ ]:
# Porcentaje de valores faltantes
(df.isna().sum() / len(df) * 100).round(2)

Preguntas guía:

¿Qué columnas tienen más valores faltantes?

¿Esos valores son críticos (por ejemplo, coordenadas o años)?

4. Manejo de valores faltantes (NaN)

In [ ]:
# Eliminar filas sin coordenadas
df = df.dropna(subset=['decimalLatitude', 'decimalLongitude'])
print("Filas después de eliminar sin coordenadas:", len(df))

# Reemplazar NaN en texto con 'Desconocido'
df = df.fillna({'stateProvince': 'Desconocido', 'country': 'Desconocido'})


5. Validación de coordenadas

In [ ]:
# Verificar si hay coordenadas fuera de rango
invalid_coords = df[
    (~df['decimalLatitude'].between(-90, 90)) |
    (~df['decimalLongitude'].between(-180, 180))
]

print("Registros con coordenadas fuera de rango:", len(invalid_coords))


In [ ]:
# Filtrar solo coordenadas válidas
df_valid = df[
    (df['decimalLatitude'].between(-90, 90)) &
    (df['decimalLongitude'].between(-180, 180))
]


*Ejercicio:
¿Cuántos registros eliminaste por tener coordenadas inválidas?
¿Dónde podrían haberse generado esos errores (por ejemplo, carga manual o fallos de GPS)?*

6. Limpieza y estandarización de fechas

In [ ]:
# Convertir columna 'year' a tipo numérico
df_valid['year'] = pd.to_numeric(df_valid['year'], errors='coerce')

# Eliminar registros con años fuera de rango razonable
df_valid = df_valid[(df_valid['year'] >= 1800) & (df_valid['year'] <= 2025)]

df_valid['year'].describe()


*Reto:
Filtra solo los registros posteriores al año 2000 para análisis contemporáneo.*

7. Limpieza de nombres científicos

In [ ]:
# Revisar valores únicos del nombre científico
df_valid['scientificName'].value_counts().head(10)

In [ ]:
# Eliminar espacios extra y caracteres especiales
df_valid['scientificName'] = df_valid['scientificName'].str.strip()
df_valid['scientificName'] = df_valid['scientificName'].str.replace('[^a-zA-Z\s]', '', regex=True)


Actividad:
Busca registros con diferencias menores de escritura (“Dasypus novemcinctus” vs “Dasypus novem cinctus”)
y corrígelos manualmente con replace().

In [ ]:
df_valid['scientificName'] = df_valid['scientificName'].replace({
    'Dasypus novem cinctus': 'Dasypus novemcinctus'
})


8. Eliminar duplicados

In [ ]:
# Verificar duplicados (basados en coordenadas y año)
duplicados = df_valid.duplicated(subset=['decimalLatitude', 'decimalLongitude', 'year'])
print("Duplicados detectados:", duplicados.sum())

# Eliminar duplicados
df_valid = df_valid.drop_duplicates(subset=['decimalLatitude', 'decimalLongitude', 'year'])
print("Filas finales:", len(df_valid))


9. Verificación final de calidad

In [ ]:
print("Filas:", len(df_valid))
print("Columnas:", df_valid.shape[1])
print("Fechas:", df_valid['year'].min(), "-", df_valid['year'].max())
df_valid.info()


*Pregunta:
¿Tu dataset ahora tiene datos completos, sin duplicados y dentro de los rangos esperados?*

10. Exportar el dataset limpio

In [ ]:
df_valid.to_csv('gbif_dasypus_limpio.csv', index=False)
print("Archivo limpio guardado: gbif_dasypus_limpio.csv")


# 🧠 Mini reto: Limpieza completa de tu propio dataset

1. Carga el dataset de la especie que descargaste desde GBIF.  
2. Revisa los valores faltantes y elimina los registros sin coordenadas.  
3. Corrige coordenadas fuera de rango.  
4. Filtra solo años válidos (1800–2025).  
5. Limpia los nombres científicos (espacios, caracteres, duplicados).  
6. Elimina registros duplicados.  
7. Exporta tu archivo limpio como `dataset_especie_limpio.csv`.

💬 *Preguntas de reflexión:*
- ¿Qué porcentaje de registros se perdió durante la limpieza?  
- ¿Qué tipo de errores eran más comunes?  
- ¿Qué estrategias usarías para automatizar esta limpieza en el futuro?


En este bloque aprendiste a:
✅ Detectar errores comunes en datos biológicos.  
✅ Aplicar limpieza sistemática con Pandas (NaN, duplicados, rangos).  
✅ Estandarizar nombres y fechas.  
✅ Exportar datasets validados y listos para análisis.  

🚀 En el siguiente bloque (Bloque 7) realizaremos un **análisis exploratorio y visualización** de estos datos limpios con Pandas, Matplotlib y Seaborn.
